# Use word2vec and random forest to tagging the question

(It not effective, but it's a good practice for me to use word2vec and random forest)

```
Hamming Loss:  0.01
F1 Score:  0.03015075376884422
```

In [3]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mysql.connector
import json

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='127.0.0.1',
    port=13306,
    user='root',
    password='root',
    database='pyml'
)

# Read the table data using pandas
query = """SELECT vi.hash_id, vi.contents as question_content, JSON_UNQUOTE(JSON_EXTRACT(tags, '$[*].slug')) AS slug FROM viblo_interview vi"""
df_ques = pd.read_sql(query, connection)

query_ans = """SELECT va.hash_id, va.contents, va.question_id FROM viblo_answer va"""
df_answer = pd.read_sql(query_ans, connection)

# Close the database connection
connection.close()


def filter_slug(x):
    if x is None:
        return ''
    t = str(x).replace("b'", "").replace("'", "")
    t1 = json.loads(t)

    return t1
df_ques['slug_filter'] = df_ques['slug'].apply(filter_slug)
print(df_ques['slug_filter'][0])


/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_92791/1993668918.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ques = pd.read_sql(query, connection)
/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_92791/1993668918.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_answer = pd.read_sql(query_ans, connection)


['javascript']


In [4]:

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

Y = df_ques['slug_filter']
print(Y[100:110])

mlb = MultiLabelBinarizer()
Y = mlb.fit_transform(Y)


print('y_test', Y[100])
t = mlb.inverse_transform(Y[100:101])
print('t', t)
print('class', mlb.classes_)
   
X = df_ques['question_content']
print(len(X))

100          [java]
101         [mysql]
102          [java]
103       [general]
104          [java]
105       [general]
106       [general]
107          [java]
108    [javascript]
109    [javascript]
Name: slug_filter, dtype: object
y_test [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
t [('java',)]
class ['agile' 'ai' 'amazon-web-services' 'android-os' 'angularjs'
 'artificial-intelligence' 'asp-net' 'attack-techniques' 'aws-lambda'
 'backend-development' 'big-data' 'business-analyst' 'c' 'c-2' 'c-lang'
 'chatbot' 'clean-code' 'cloud-computing' 'cloud-security'
 'computer-network' 'computer-vision' 'content-creator' 'cryptography'
 'css' 'cyber-security' 'dart' 'data-science'
 'data-structures-and-algorithms' 'database' 'deep-learning'
 'design-pattern' 'devops' 'distribute-system' 'docker'
 'external-communicati

## Build word2vec for sentences

In [5]:
from underthesea import word_tokenize

def segmentation(text):
    try:
        return word_tokenize(text, format="text")
    except (TypeError, AttributeError):
        return ''

def split_words(post):
    texts = segmentation(post)
    try:
        t = [str(x.strip('0123456789%@$.,=+-!;/()*"&^:#|\n\t\' ').lower()) for x in texts.split()]
        filtered_list = [item for item in t if item != ""]
        return filtered_list
    except TypeError:
        return []
    
X_1 = X.apply(split_words)

In [6]:
print(X_1[:2])

0    [thế, nào, là, abstract, syntax, tree, ?, so, ...
1    [nếu, các, kỹ_thuật, thường, được, sử_dụng, để...
Name: question_content, dtype: object


In [7]:
from gensim.models import Word2Vec
sentences = X_1

# Build the word2vec model
model = Word2Vec(X_1.values.tolist(), min_count=1)

In [8]:
# Convert the dictionary to a list of key-value pairs and slice the list
first_five = list(model.wv.key_to_index.items())[20:35]

# Print the first 5 elements
print(first_five)

[('khác', 20), ('cách', 21), ('như', 22), ('khi', 23), ('cho', 24), ('có_thể', 25), ('với', 26), ('nhau', 27), ('những', 28), ('giải_thích', 29), ('nêu', 30), ('`', 31), ('khác_biệt', 32), ('tại_sao', 33), ('hãy', 34)]


Ở đây mình thứ 2 loại cơ bản, là mean hoặc flatten. Mean mình thấy được dùng phổ biến hơn, vì nó đảm bảo lenght của vector đầu ra là cố định, đỡ mất công thêm bước pad sequence, còn flatten thì không.

In [9]:
import numpy as np

def flattern_embedding(words):
    embeddings = [model.wv[word] for word in words if word in model.wv.index_to_key]
    mean_embedding = np.array(embeddings).flatten()
    return mean_embedding

def mean_embedding(words):
    embeddings = [model.wv[word] for word in words if word in model.wv.index_to_key]
    mean_embedding = np.mean(embeddings, axis=0)
    return mean_embedding

X_2 = X_1.apply(flattern_embedding)
X_3 = X_1.apply(mean_embedding)


In [27]:
print('X_2:', len(X_3[2]), len(X_3[3]))

X_2: 100 100


In [37]:
from keras_preprocessing import sequence
import numpy as np

# Pad or truncate the list to a fixed length of 5
maxlen = 100
X_padded = sequence.pad_sequences(X_2, maxlen=maxlen, dtype='float32', padding='post', truncating='post', value=0.0)

In [38]:
print(X_padded[5], X_3[5])

[-3.62489432e-01  5.99958897e-01 -7.07902163e-02  3.92022766e-02
 -7.48139545e-02 -7.99176455e-01  3.24479848e-01  9.98091459e-01
 -3.68338943e-01 -8.36061239e-01 -4.51165624e-02 -6.78606629e-01
  2.70312786e-01  2.09343567e-01 -2.55380362e-01 -7.91635096e-01
  4.72306699e-01 -9.26185906e-01 -9.83465537e-02 -1.33544910e+00
  3.68279546e-01  2.78866351e-01  6.20059609e-01 -9.69156846e-02
 -1.79394603e-01 -1.81375407e-02 -1.71344765e-02 -1.95301250e-01
 -5.29945552e-01 -2.03762844e-01  5.95222592e-01  3.66457850e-02
  1.77748371e-02 -5.84770203e-01  5.13280109e-02  9.82236087e-01
  3.08333248e-01 -6.51353359e-01 -5.93731403e-01 -6.86787128e-01
  3.52721184e-01 -6.99060023e-01 -5.00993840e-02 -1.77912056e-01
  8.75916421e-01 -8.70144218e-02 -3.30352992e-01 -5.39628528e-02
  2.81746119e-01  8.93776640e-02  6.07260644e-01 -2.94409007e-01
 -1.00750022e-01 -4.42870140e-01 -5.36435544e-01  1.20721132e-01
  1.61158085e-01 -3.77101675e-02 -5.47625065e-01 -5.00631556e-02
  4.59412098e-01  7.40608

In [39]:
def encode(texts = [], maxlen = maxlen):
    encoded_texts = []
    for text in texts:
        words = split_words(text)
        encoded_texts.append(flattern_embedding(words))
        # Pad or truncate the list to a fixed length of 5
    X_padded = sequence.pad_sequences(encoded_texts, maxlen=maxlen, dtype='float32', padding='post', truncating='post', value=0.0)
    return X_padded

In [62]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

X_list = X_padded.tolist()
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X_list, Y, test_size=0.2, random_state=42)

In [63]:
print(len(X_train), len(X_train[0]), len(Y_train), len(Y_train[0]))
print(Y_train[:2])

2893 100 2893 109
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0]]


In [64]:
def get_list_container(class_list):
    result = []
    for i in class_list:
        if i == 1:
            result.append(i)
    return len(result)

for y in Y_train:
    print(get_list_container(y))

t = [get_list_container(y) for y in Y_train]
print(t[:10])


2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
1
3
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
2
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1


In [69]:
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

# ker = SVC(kernel='linear')
ker = RandomForestClassifier(n_estimators=100)
multi_output_svc = MultiOutputClassifier(ker)

# Fit the classifier on the training data
multi_output_svc.fit(X_train, Y_train)

In [67]:
from sklearn.metrics import hamming_loss, f1_score

prediction = multi_output_svc.predict(X_test)
# print('Prediction: ', prediction[4:12])
print('Hamming Loss: ', round(hamming_loss(Y_test, prediction),2))
print('F1 Score: ', f1_score(Y_test, prediction, average='micro'))

print('F1 Score: ', f1_score(Y_test, prediction, average=None))

Hamming Loss:  0.01
F1 Score:  0.005141388174807198
F1 Score:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/Users/ngocp/.pyenv/versions/3.10.3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [68]:
X_test_free = [
    'Phân biệt mạng Object Detection 2 stage và 1 stage',
    'Những lỗ hổng thường gặp với API',
    'IAM là gì, IAM có vai trò gì trong việc đảm bảo an toàn hệ thống AWS',
    'Có bao nhiêu loại lỗi sai trong PHP?',
    'Git Submodule trong trường hợp nào ?',
    'Có sự thừa kế theo cấp bậc giữa các module trong không? Giải thích.',
    'Sự khác biệt giữa từ khóa break và continue trong Java?',
    'Trình bày về Output Buffering trong PHP?',
    'RESTful API là gì?',
    'Viết tắt của php có nghĩa là gì ?',
    'Phân biệt POST và GET trong php?',
    'Cờ HttpOnly có tác dụng gì cho cookie?',
    'so sánh sự khác nhau giữa Mysql và MongoDB',
    'Tại sao phải sử dụng hàm khởi tạo?',
    'Hash trong Ruby có thể lưu trữ các loại dữ liệu nào?',
]

X_test_embed = encode(X_test_free)

# Predict the labels for the test data
Y_pred_free = multi_output_svc.predict(encode(X_test_free))

Y_pred_free_class = mlb.inverse_transform(Y_pred_free)

for index, tags in enumerate(Y_pred_free_class):
    print(X_test_free[index][:100], '->', tags)

Phân biệt mạng Object Detection 2 stage và 1 stage -> ()
Những lỗ hổng thường gặp với API -> ()
IAM là gì, IAM có vai trò gì trong việc đảm bảo an toàn hệ thống AWS -> ()
Có bao nhiêu loại lỗi sai trong PHP? -> ()
Git Submodule trong trường hợp nào ? -> ()
Có sự thừa kế theo cấp bậc giữa các module trong không? Giải thích. -> ()
Sự khác biệt giữa từ khóa break và continue trong Java? -> ()
Trình bày về Output Buffering trong PHP? -> ()
RESTful API là gì? -> ()
Viết tắt của php có nghĩa là gì ? -> ()
Phân biệt POST và GET trong php? -> ()
Cờ HttpOnly có tác dụng gì cho cookie? -> ()
so sánh sự khác nhau giữa Mysql và MongoDB -> ()
Tại sao phải sử dụng hàm khởi tạo? -> ()
Hash trong Ruby có thể lưu trữ các loại dữ liệu nào? -> ()


In [ ]:
# Kết luận.
Tính ra nếu tự train word2vec từ chính X_train thì kết quả khá tệ, F1 score chỉ đạt 0.15, 
một số câu hỏi cơ bản không trả lời được, có thể do dữ liệu train không đủ lớn, như git chỉ tầm 10 câu hỏi, nên 
không đủ thể hiện vector áng xạ, model không catch up được các đặc tính của các từ khóa trong câu.

